In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
import numpy as np
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn as nn
import os
from PIL import Image

import json
import numpy as np
import matplotlib.pyplot as plt
from pycocotools.coco import COCO


In [3]:
if torch.cuda.is_available():
    print("The code will run on GPU.")
else:
    print("The code will run on CPU. Go to Edit->Notebook Settings and choose GPU as the hardware accelerator")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.get_device_name(0))
# print(torch.cuda.memory_usage(0))

The code will run on GPU.
Tesla V100-PCIE-32GB


In [4]:
ROOT_PATH = '/dtu/datasets1/02514/data_wastedetection'
anns_file_path = ROOT_PATH + '/' + 'annotations.json'

In [5]:
%matplotlib inline
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# Read annotations

with open(anns_file_path, 'r') as f:
    dataset = json.loads(f.read())

categories = dataset['categories']
anns = dataset['annotations']
imgs = dataset['images']
nr_cats = len(categories)
nr_annotations = len(anns)
nr_images = len(imgs)

# Load categories and super categories
cat_names = []
super_cat_names = []
super_cat_ids = {}
super_cat_last_name = ''
nr_super_cats = 0
for cat_it in categories:
    cat_names.append(cat_it['name'])
    super_cat_name = cat_it['supercategory']
    # Adding new supercat
    if super_cat_name != super_cat_last_name:
        super_cat_names.append(super_cat_name)
        super_cat_ids[super_cat_name] = nr_super_cats
        super_cat_last_name = super_cat_name
        nr_super_cats += 1
super_cat_names = ["background"]+super_cat_names

print('Number of super categories:', nr_super_cats)
print('Number of categories:', nr_cats)
print('Number of annotations:', nr_annotations)
print('Number of images:', nr_images)

Number of super categories: 28
Number of categories: 60
Number of annotations: 4784
Number of images: 1500


### Linking the regulear categories to super categories

In [6]:
cat_ids_2_supercat_ids = {}
for cat in categories:
    cat_ids_2_supercat_ids[cat['id']+1] = super_cat_ids[cat['supercategory']]+1
    
class label(object):
    def __init__(self,cat_ids_2_supercat_ids):
        self.cat_ids_2_super_cat_ids = cat_ids_2_supercat_ids
    def __call__(self,sample):
        for i in range(len(sample)):
            sample[i]["category_id"] = cat_ids_2_supercat_ids[sample[i]["category_id"]]
        return sample

In [7]:
train_transform = transforms.Compose([transforms.Resize((500, 500)),
                                     ])

In [8]:
def get_iou(data, ss,x_scale,y_scale):
    gt = []
    inside_gt = []
    background = []
    
    for objects in range(len(data)):
        gt.append(data[objects]["bbox"])
        x11 = gt[objects][0]*x_scale
        y11 = gt[objects][1]*y_scale
        x12 = (gt[objects][2]*x_scale)+x11
        y12 = (gt[objects][3]*y_scale)+y11
        add_background = False
        for index,candidates in enumerate(ss):

            x21 = ss[index][0]
            y21 = ss[index][1]
            x22 = ss[index][2]+x21
            y22 = ss[index][3]+y21
            
            box1 = torch.tensor([[x11, y11, x12, y12]], dtype=torch.float)
            box2 = torch.tensor([[x21, y21, x22, y22]], dtype=torch.float)
            iou = float(bops.box_iou(box1, box2).cpu())
            if iou > 0.5:
                inside_gt.append([[x21,y21,x22-x21,y22-y21],data[objects]["category_id"]])
                add_background = True
                
            if (add_background == True) & (iou < 0.20):
                background.append([[x21,y21,x22-x21,y22-y21],0])
                add_background = False
                
                
    return inside_gt+background

In [9]:
class CocoDetection(torch.utils.data.Dataset):
    """`MS Coco Detection <http://mscoco.org/dataset/#detections-challenge2016>`_ Dataset.

    Args:
        root (string): Root directory where images are downloaded to.
        annFile (string): Path to json annotation file.
        transform (callable, optional): A function/transform that  takes in an PIL image
            and returns a transformed version. E.g, ``transforms.ToTensor``
        target_transform (callable, optional): A function/transform that takes in the
            target and transforms it.
    """

    def __init__(self, root, annFile, transform=None, target_transform=None):
        from pycocotools.coco import COCO
        self.root = root
        self.coco = COCO(annFile)
        self.ids = list(self.coco.imgs.keys())
        self.transform = transform
        self.target_transform = target_transform

    def __getitem__(self, index):
        """
        Args:
            index (int): Index

        Returns:
            tuple: Tuple (image, target). target is the object returned by ``coco.loadAnns``.
        """
        coco = self.coco
        self.img_id = self.ids[index]
        ann_ids = coco.getAnnIds(imgIds=self.img_id)
        target = coco.loadAnns(ann_ids)

        path = coco.loadImgs(self.img_id)[0]['file_name']

        img = Image.open(os.path.join(self.root, path)).convert('RGB')
        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            
            target = self.target_transform(target)
        
        return img, target


    def __len__(self):
        return len(self.ids)

    def __repr__(self):
        fmt_str = 'Dataset ' + self.__class__.__name__ + '\n'
        fmt_str += 'Number of datapoints: {}\n'.format(self.__len__())
        fmt_str += 'Root Location: {}\n'.format(self.root)
        tmp = '    Transforms (if any): '
        fmt_str += '{0}{1}\n'.format(tmp, self.transform.__repr__().replace('\n', '\n' + ' ' * len(tmp)))
        tmp = '    Target Transforms (if any): '
        fmt_str += '{0}{1}'.format(tmp, self.target_transform.__repr__().replace('\n', '\n' + ' ' * len(tmp)))
        return fmt_str


In [10]:
batch_size = 1
coco = COCO(anns_file_path)

dataset = CocoDetection(ROOT_PATH,anns_file_path,target_transform=label(cat_ids_2_supercat_ids),transform=train_transform)

trainsize = int(0.8*len(dataset))
testsize = len(dataset)-trainsize

train_data,test_data = torch.utils.data.random_split(dataset,[trainsize,testsize])
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=3,collate_fn=lambda x: x)
test_loader  = DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=3,collate_fn=lambda x: x)

loading annotations into memory...
Done (t=0.05s)
creating index...
index created!
loading annotations into memory...
Done (t=0.14s)
creating index...
index created!


In [11]:
import cv2
import matplotlib.pyplot as plt 
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from PIL import Image
import numpy as np
from matplotlib.patches import Rectangle
import torchvision.ops.boxes as bops

In [12]:
##plt.imshow(image)

#for index,rect in enumerate(inside):
#    plt.gca().add_patch(Rectangle((rect[0],rect[1]),rect[2],rect[3],
#                        edgecolor='red',
#                        facecolor='none',
#                        lw=1))
#    for i in range(len(data)):
#        plt.gca().add_patch(Rectangle((data[i]["bbox"][0]*x_scale,data[i]["bbox"][1]*y_scale),data[i]["bbox"][2]*x_scale,data[i]["bbox"][3]*y_scale,
#                            edgecolor='blue',
#                            facecolor='none',
#                            lw=1))


### Creating model and training loop

In [13]:
batch_size = 1
coco = COCO(anns_file_path)

dataset = CocoDetection(ROOT_PATH,anns_file_path,target_transform=label(cat_ids_2_supercat_ids),transform=train_transform)

trainsize = int(0.8*len(dataset))
testsize = len(dataset)-trainsize

train_data,test_data = torch.utils.data.random_split(dataset,[trainsize,testsize])
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=3,collate_fn=lambda x: x)
test_loader  = DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=3,collate_fn=lambda x: x)

loading annotations into memory...
Done (t=0.05s)
creating index...
index created!
loading annotations into memory...
Done (t=0.05s)
creating index...
index created!


In [14]:
from torchvision import models
import torch.optim as optim

In [40]:
model = models.mobilenet_v3_large()

### Changing the last layer to be 10 instead of 1000
model.classifier[3] = nn.Linear(1280,60,bias=True)

model = model.to(device)

In [30]:
#We define the training as a function so we can easily re-use it.
def train(model, optimizer, num_epochs=1):
    #def loss_fun(output, target):
    #    return F.nll_loss(torch.log(output), target)
    loss_fun = nn.CrossEntropyLoss()
    out_dict = {'train_acc': [],
              'test_acc': [],
              'train_loss': [],
              'test_loss': []}
    ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
  
    for epoch in tqdm(range(num_epochs), unit='epoch'):
        model.train()
        #For each epoch
        train_correct = 0
        train_loss = []
        crop_images = []
        #a = np.asarray(next(iter(train_loader)))
        
        for minibatch_number, images in enumerate(train_loader):
            
            for index, (image,data) in enumerate(images):
                ### Running selective search on image
                np_image = np.asarray(image)
                ss.setBaseImage(np_image)
                ss.switchToSelectiveSearchFast()
                rects = ss.process()
                
                og_height = coco.loadImgs(data[0]["image_id"])[0]["height"]
                og_width  = coco.loadImgs(data[0]["image_id"])[0]["width"]
                x_scale = 500/og_width
                y_scale = 500/og_height
                
                crop_rects_and_labels = get_iou(data,rects,x_scale,y_scale)
                
                for rects_and_labels in crop_rects_and_labels:
                    (x1,y1,w,h),(label) = rects_and_labels
                    x2 = x1+w
                    y2 = y1+h
                    crop_images.append(tuple([transforms.functional.pil_to_tensor(transforms.functional.resize(image.crop((x1,y1,x2,y2)),(60,60))).to(dtype=torch.float32),label]))                                     
            
            ### Having enough data for a training loop   
            #crop_loader = DataLoader(crop_images,batch_size=batch_size,shuffle=True, num_workers=3)
            trainsize = int(0.8*len(crop_images))
            testsize = len(crop_images)-trainsize

            crop_train_data,crop_test_data = torch.utils.data.random_split(crop_images,[trainsize,testsize])
            crop_train_loader = DataLoader(crop_train_data, batch_size=batch_size, shuffle=True, num_workers=3)
            crop_test_loader  = DataLoader(crop_test_data, batch_size=batch_size, shuffle=True, num_workers=3)  
          
            model.train()
            #For each epoch
            train_correct = 0
            train_loss = []
            for minibatch_no, (data, target) in tqdm(enumerate(crop_train_loader), total=len(crop_train_loader)):
                data, target = data.to(device), target.to(device)
                #Zero the gradients computed for each weight
                optimizer.zero_grad()
                #Forward pass your image through the network
                output = model(data)
                #Compute the loss
                loss = loss_fun(output, target)            
                #Backward pass through the network
                loss.backward()
                #Update the weights
                optimizer.step()

                train_loss.append(loss.item())
                #Compute how many were correctly classified
                predicted = output.argmax(1)
                train_correct += (target==predicted).sum().cpu().item()
                
            #Comput the test accuracy
            test_loss = []
            test_correct = 0
            model.eval()
            for data, target in crop_test_loader:
                data, target = data.to(device), target.to(device)
                with torch.no_grad():
                    output = model(data)
                test_loss.append(loss_fun(output, target).cpu().item())
                predicted = output.argmax(1)
                test_correct += (target==predicted).sum().cpu().item()
            out_dict['train_acc'].append(train_correct/len(crop_train_data))
            out_dict['test_acc'].append(test_correct/len(crop_test_data))
            out_dict['train_loss'].append(np.mean(train_loss))
            out_dict['test_loss'].append(np.mean(test_loss))
            print(f"Loss train: {np.mean(train_loss):.3f}\t test: {np.mean(test_loss):.3f}\t",
                  f"Accuracy train: {out_dict['train_acc'][-1]*100:.1f}%\t test: {out_dict['test_acc'][-1]*100:.1f}%")
        return out_dict


In [27]:
optimizer = optim.Adam(model.parameters(),lr = 0.001)

In [31]:
crop_images,crop_train_loader, crop_test_loader= train(model,optimizer)

  0%|          | 0/1 [00:00<?, ?epoch/s]

1


  0%|          | 0/42 [00:00<?, ?it/s]

Loss train: 2.413	 test: 720.558	 Accuracy train: 38.1%	 test: 0.0%
1


  0%|          | 0/55 [00:00<?, ?it/s]

KeyboardInterrupt: 